In [1]:
#my approch
import os
import cv2
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity as ssim
from skimage.measure import shannon_entropy

import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.measure import shannon_entropy

def compute_metrics(original, enhanced):
    """
    Compute PSNR, SSIM, and Shannon Entropy between the original and enhanced images.
    
    Parameters:
        original (np.array): Original image (BGR, uint8).
        enhanced (np.array): Enhanced image (BGR, uint8).
    
    Returns:
        tuple: (PSNR, SSIM, Entropy)
    """
    # Ensure both images have the same size (resize enhanced if necessary)
    if original.shape != enhanced.shape:
        enhanced = cv2.resize(enhanced, (original.shape[1], original.shape[0]))
    
    # PSNR computation
    psnr_value = cv2.PSNR(original, enhanced)
    
    # Determine an appropriate win_size for SSIM based on the image dimensions.
    h, w = original.shape[:2]
    min_dim = min(h, w)
    
    # If the smallest dimension is less than 3 pixels, SSIM is unreliable.
    if min_dim < 3:
        ssim_value = np.nan
    else:
        # Choose a win_size that is odd and does not exceed min_dim.
        if min_dim < 7:
            win_size = min_dim if (min_dim % 2 == 1) else (min_dim - 1)
        else:
            win_size = 7  # default win_size for larger images
        
        # Compute SSIM using the computed win_size and specifying channel_axis=-1 for color images.
        try:
            ssim_value, _ = ssim(original, enhanced, full=True, channel_axis=-1, win_size=win_size)
        except ValueError as e:
            print(f"Error computing SSIM for image of size ({h}, {w}) with win_size={win_size}: {e}")
            ssim_value = np.nan

    # Compute Shannon entropy on the enhanced image.
    # Convert the enhanced image to grayscale.
    enhanced_gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
    entropy_value = shannon_entropy(enhanced_gray)
    
    return psnr_value, ssim_value, entropy_value



def process_folders(original_folder, enhanced_folder, output_csv=r"D:\mit-5k-subset\cidnet_enhancement_metrics_parte_generalization.csv"):
    valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')
    results = []

    for filename in os.listdir(original_folder):
        if filename.lower().endswith(valid_extensions):
            orig_path = os.path.join(original_folder, filename)

            # Remove 'normal' prefix and add 'low' prefix
            basename = filename.replace('normal', 'low', 1)  # replace only the first occurrence
            enh_path = os.path.join(enhanced_folder, basename)

            if not os.path.exists(enh_path):
                print(f"Enhanced image for {filename} not found (expected {basename}). Skipping.")
                continue

            original_img = cv2.imread(orig_path)
            enhanced_img = cv2.imread(enh_path)

            if original_img is None or enhanced_img is None:
                print(f"Error reading {filename}. Skipping.")
                continue

            psnr_val, ssim_val, entropy_val = compute_metrics(original_img, enhanced_img)
            results.append({
                'Filename': filename,  # you can also use a common name like basename if you prefer
                'CIDNet PSNR': psnr_val,
                'CIDNet SSIM': ssim_val,
                'CIDNet Entropy': entropy_val
            })
            print(f"Processed {filename}: PSNR={psnr_val:.2f}, SSIM={ssim_val:.3f}, Entropy={entropy_val:.2f}")

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")
    return df




    # Replace these with the paths to your folders.
original_folder = r"D:\mit-5k-subset\raw"
enhanced_folder = r"D:\mit-5k-subset\e_enhanced_cidnet_generalize"
    
    # Process the folders and save metrics to a CSV.
metrics_df2 = process_folders(original_folder, enhanced_folder)


Processed a0034-LSYD4O2202.jpg: PSNR=16.28, SSIM=0.336, Entropy=7.53
Processed a0035-dgw_048.jpg: PSNR=12.96, SSIM=0.479, Entropy=7.54
Processed a0291-IMG_0115.jpg: PSNR=17.68, SSIM=0.316, Entropy=7.51
Processed a0436-IMG_2583.jpg: PSNR=13.56, SSIM=0.656, Entropy=7.78
Processed a0452-IMG_1646.jpg: PSNR=13.01, SSIM=0.643, Entropy=6.72
Processed a0463-jmac_DSC2316.jpg: PSNR=11.68, SSIM=0.469, Entropy=7.08
Processed a0631-NKIM_MG_6442.jpg: PSNR=13.63, SSIM=0.648, Entropy=6.60
Processed a0648-IMG_5085.jpg: PSNR=13.47, SSIM=0.551, Entropy=7.67
Processed a0712-_DSC8911.jpg: PSNR=13.50, SSIM=0.622, Entropy=7.56
Processed a0752-20061213_134314__MG_3708.jpg: PSNR=16.80, SSIM=0.516, Entropy=7.00
Processed a0767-jn_20070824_0165.jpg: PSNR=11.89, SSIM=0.259, Entropy=7.81
Processed a0770-050703_161554__I2E9266.jpg: PSNR=22.06, SSIM=0.695, Entropy=7.10
Processed a0775-kme_423.jpg: PSNR=9.69, SSIM=0.320, Entropy=7.76
Processed a0811-20051224_165428__MG_0953.jpg: PSNR=17.12, SSIM=0.875, Entropy=6.43
P

In [2]:
import matplotlib.pyplot as plt
# Compute Averages
average_values = metrics_df2.mean(numeric_only=True)
print("\nAverage Values of CIDNet Generalization approach:")
print(average_values)


Average Values of CIDNet Generalization approach:
CIDNet PSNR       13.393217
CIDNet SSIM        0.505696
CIDNet Entropy     7.330030
dtype: float64
